Drive link  sample footage for video testing - [VideoDownlodLink](https://drive.google.com/drive/folders/16KqOksNhvhnopUC0oQ_FjuxeIvz2E5lI?usp=sharing)

In [ ]:
!pip install opencv-python torch gradio

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0


In [ ]:
!pip install ultralytics==8.0.196

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 21.4 MB/s eta 0:00:00


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="nfRUp06xCyFRegBxM8Mr")
project = rf.workspace("saras-9qbcz").project("emergency-vehicle-detection-t2tck-urnyv")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Emergency-Vehicle-Detection-1 in yolov8:: 100%|██████████| 1615/1615 [00:00<00:00, 3845.41it/s]


In [ ]:
!yolo train model=yolov8m.pt data=/content/Emergency-Vehicle-Detection-1/data.yaml epochs=3 imgsz=640

100% 49.7M/49.7M [00:00<00:00, 135MB/s]
New https://pypi.org/project/ultralytics/8.2.75 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/Emergency-Vehicle-Detection-1/data.yaml, epochs=3, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffe

In [ ]:
import cv2
import torch
import gradio as gr
from ultralytics import YOLO
import numpy as np

# pre trained model for pedestrian , car etc detection and for comparison with custom trained model
model_yolov8x = YOLO('yolov8x.pt')  # Using a larger, more accurate model

# new model trained on ambulance and fire truck
model_new = YOLO('runs/detect/train/weights/best.pt')


traffic_light_position = None

def process_video(video_path):
    global traffic_light_position
    traffic_light_position = None
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))


    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_video_path = 'output.mp4'
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # pretrained to detect cars, persons, buses, and traffic lights
        results_yolov8x = model_yolov8x(frame)

        # new model to detect ambulance and fire truck
        results_new = model_new(frame)

        for r in results_yolov8x:
            boxes = r.boxes
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls = int(box.cls[0])
                conf = float(box.conf[0])
                class_name = model_yolov8x.names[cls]

                if class_name in ['car', 'person', 'bus', 'traffic light']:
                    color = (0, 255, 0)


                    label = class_name
                    label += f' {conf:.2f}'
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
                    cv2.rectangle(frame, (x1, y1 - 20), (x1 + w, y1), color, -1)
                    cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)


                if class_name == 'traffic light':
                    traffic_light_position = (x1 + (x2 - x1) // 2, y1 + (y2 - y1) // 2)
                    # reference point for traffic light
                    cv2.circle(frame, traffic_light_position, 5, (0, 255, 255), -1)  # Yellow point

        for r in results_new:
            boxes = r.boxes
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls = int(box.cls[0])
                conf = float(box.conf[0])
                class_name = model_new.names[cls]

                if class_name in ['ambulance', 'fire truck']:
                    color = (0, 0, 255)


                    label = class_name
                    label += f' {conf:.2f}'
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
                    cv2.rectangle(frame, (x1, y1 - 20), (x1 + w, y1), color, -1)
                    cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)


                    emergency_vehicle_position = (x1 + (x2 - x1) // 2, y1 + (y2 - y1) // 2)


                    if traffic_light_position is not None:
                        cv2.line(frame, emergency_vehicle_position, traffic_light_position, (255, 0, 0), 2)  # Blue line

        out.write(frame)

    cap.release()
    out.release()

    return 'output.mp4'
iface = gr.Interface(
    fn=process_video,
    inputs=gr.Video(label="Input Traffic Video"),
    outputs=gr.Video(label="Processed Video"),
    title=" Emergency Vehicle Recognition using yolov8 pretrained and custom trained model",
    description="Upload a traffic video for emergency vehicle detection  \n  Note : if your are facing (Video does not have browser-compatible container or codec. Converting to mp4) issue please download the ouput.mp4(file size should be more than 10mb ) file from files ",
    allow_flagging="never"
)

iface.launch(debug=True)